# Especialização em Ciência de Dados - PUC-Rio
# Machine Learning
## Automatização de Fluxos com Pipelines / Salvando e carregando modelos

# Automatização de Fluxos com Pipelines

Existem fluxos de trabalho padrão no aprendizado de máquina aplicado. O scikit-learn do Python fornece um utilitário Pipeline para ajudar a automatizar os fluxos de trabalho de aprendizado de máquina. Os pipelines permitem que uma sequência linear de transformações de dados seja encadeada, culminando em um processo de modelagem que pode ser avaliado.

O objetivo é garantir que todas as etapas do pipeline sejam restritas aos dados disponíveis para a avaliação, como o conjunto de dados de treinamento ou cada fold do procedimento de validação cruzada. 

Você pode aprender mais sobre pipelines no scikit-learn lendo a seção Pipeline do guia do usuário e na documentação da API para as classes Pipeline e FeatureUnion e o módulo de pipeline:
* http://scikit-learn.org/stable/modules/pipeline.html 
* http://scikit-learn.org/stable/modules/classes.html#module-sklearn.pipeline

## Importação dos dados
Usaremos a mesma base de dados que trabalhamos anteriormente, o dataset Pima Indians Diabetes. O trecho de código a seguir é idêntico ao que fizemos em laboratórios anteriores e serve apenas para importar os dados e separá-los em entradas (X) e saídas (Y).

In [0]:
# Carrega arquivo csv usando Pandas usando uma URL

# Importa todo o pacote Pandas
import pandas as pd

# Importa a função datasets
from sklearn import datasets

# Informa a URL de importação do dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

# Informa o cabeçalho das colunas
colunas = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']

# Lê o arquivo utilizando as colunas informadas
dataset = pd.read_csv(url, names=colunas, skiprows=0, delimiter=',')

# Pega apenas os dados do dataset e guardando em um array
array = dataset.values

# Separa o array em variáveis preditoras (X) e variável target (Y)
X = array[:,0:8]
Y = array[:,8]

## Pipeline de preparação e modelagem de dados

Uma armadilha comum no aprendizado de máquina aplicado é o "vazamento" de dados do conjunto de dados de treinamento para o conjunto de dados de teste. Para evitar essa armadilha, precisamos de uma forte separação de treinamento e teste, incluindo a preparação de dados. Na preparação de dados é fácil "vazar" o conhecimento de todo o conjunto de dados de treinamento para o algoritmo. Por exemplo, preparar os dados usando normalização ou padronização em todo o conjunto de dados de treinamento antes do aprendizado não seria recomendado, porque as partições de treinamento seriam influenciados pela escala dos dados da partição do conjunto de testes.

Os pipelines garantem que a preparação de dados seja restrita a cada fold da validação cruzada. Neste exemplo, o pipeline é definido com duas etapas:

1. Padronização dos dados.
2. Aprendizado de um modelo.

O pipeline é então avaliado usando a validação cruzada 10-fold.

In [0]:
# Criação de um pipeline para padronizar os dados e criar um modelo

# Imports
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Criação do pipeline com apenas 1 modelo
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('lda', LinearDiscriminantAnalysis()))
model = Pipeline(estimators)

# Avaliação do pipeline
kfold = KFold(n_splits=10, random_state=7)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

Observe como criamos uma lista Python de etapas fornecidas ao Pipeline para o processamento dos dados. Observe também como o próprio pipeline é tratado como um estimador e é avaliado na íntegra pelo procedimento de validação cruzada.

## Extração de features e pipeline de modelagem

A extração de features é outro procedimento suscetível a vazamento de dados. Como a preparação de dados, os procedimentos de extração de features devem ser restritos aos dados do conjunto de dados de treinamento. O pipeline fornece uma ferramenta útil chamada FeatureUnion, que permite combinar os resultados de vários procedimentos de seleção e extração de features em um conjunto de dados maior no qual um modelo pode ser treinado. Toda a extração e união de features ocorre em cada fold do procedimento de validação cruzada. Neste exemplo, o pipeline é definido com quatro etapas:

1. Extração de features com análise de componentes principais (3 features).
2. Extração de features com seleção estatística (6 features).
3. União de features.
4. Aprendizado de um modelo.

O pipeline é então avaliado usando a validação cruzada 10-fold.

In [0]:
# Criação de um pipeline para extrair features dos dados e criar um modelo

# Imports
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest

# Cria a união de features
features = []
features.append(('pca', PCA(n_components=3)))
features.append(('select_best', SelectKBest(k=6)))
feature_union = FeatureUnion(features)

# Cria o pipeline com apenas 1 modelo
estimators = []
estimators.append(('feature_union', feature_union)) 
estimators.append(('logistic', LogisticRegression(solver='liblinear')))
model = Pipeline(estimators)

# Avaliação do pipeline
kfold = KFold(n_splits=10, random_state=7)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

Observe como o FeatureUnion é seu próprio pipeline que, por sua vez, é uma única etapa no pipeline final usado para alimentar a regressão logística. Isso pode fazer você pensar em como começar a incorporar pipelines dentro de pipelines.

# Salvando e Carregando Modelos

## Pickle
https://docs.python.org/2/library/pickle.html

É a maneira padrão de serializar objetos em Python, sendo possível serializar modelos de aprendizado de máquina e salvar o formato serializado em um arquivo. Posteriormente, é possível pode carregar esse arquivo para desserializar o modelo e usá-lo para fazer novas previsões.

In [0]:
# Imports
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from pickle import dump
from pickle import load

# Divide os dados em treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=7)

# Cria o modelo
modelo = LogisticRegression(solver='liblinear') 

# Treina o modelo
modelo.fit(X_train, Y_train)

# Salva o modelo no disco
filename = 'finalized_model.sav'
dump(modelo, open(filename, 'wb'))

# Algum tempo depois...
# Carrega o modelo no disco
loaded_model = load(open(filename, 'rb')) 
result = loaded_model.score(X_test, Y_test) 
print(result)

## Joblib

https://pypi.python.org/pypi/joblib
https://pythonhosted.org/joblib/generated/joblib.dump.html

A biblioteca Joblib faz parte do ecossistema SciPy e fornece utilitários para pipeline de trabalhos em Python. Fornece utilitários para salvar e carregar objetos Python que fazem uso eficiente das estruturas de dados do NumPy. Isso pode ser útil para alguns algoritmos de aprendizado de máquina que exigem muitos parâmetros ou armazenam todo o conjunto de dados (por exemplo, KNN).

In [0]:
# Imports
from sklearn.linear_model import LogisticRegression
from sklearn.externals.joblib import dump
from sklearn.externals.joblib import load

# Divide os dados em treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=7) 

# Cria o modelo
modelo = LogisticRegression(solver='liblinear')

# Treina o modelo
modelo.fit(X_train, Y_train)

# Salva o modelo no disco
filename = 'finalized_model.sav' 
dump(modelo, filename)

# Algum tempo depois...
# Carrega o modelo no disco
loaded_model = load(filename)
result = loaded_model.score(X_test, Y_test)
print(result)